## Create datasets for the Content-based Filter

This notebook builds the data we will use for creating our content based model. We'll collect the data via a collection of SQL queries from the publicly available Kurier.at dataset in BigQuery.
Kurier.at is an Austrian newsite. The goal of these labs is to recommend an article for a visitor to the site. In this lab we collect the data for training, in the subsequent notebook we train the recommender model. 

This notebook illustrates
* how to pull data from BigQuery table and write to local files
* how to make reproducible train and test splits 

In [1]:
!pip3 install tensorflow-hub==0.7.0
!pip3 install --upgrade tensorflow==1.15.3
!pip3 install google-cloud-bigquery==1.10

  Using cached google_cloud_core-0.29.1-py2.py3-none-any.whl (25 kB)
  Attempting uninstall: google-cloud-core
    Found existing installation: google-cloud-core 1.7.2
    Uninstalling google-cloud-core-1.7.2:
      Successfully uninstalled google-cloud-core-1.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-translate 3.6.0 requires google-cloud-core<3.0.0dev,>=1.3.0, but you have google-cloud-core 0.29.1 which is incompatible.
google-cloud-storage 1.42.3 requires google-cloud-core<3.0dev,>=1.6.0; python_version >= "3.6", but you have google-cloud-core 0.29.1 which is incompatible.
google-cloud-spanner 3.11.1 requires google-cloud-core<3.0dev,>=1.4.1, but you have google-cloud-core 0.29.1 which is incompatible.
google-cloud-logging 2.6.0 requires google-cloud-core<3.0.0dev,>=1.4.1, but you have google-cloud-core 0.29.1 which is incompatible.
goo

In [2]:
import os
import tensorflow as tf
import numpy as np
from google.cloud import bigquery 

PROJECT = 'qwiklabs-gcp-03-b731988fa21f' # REPLACE WITH YOUR PROJECT ID
BUCKET = 'qwiklabs-gcp-03-b731988fa21f' # REPLACE WITH YOUR BUCKET NAME
REGION = 'us-east1' # REPLACE WITH YOUR BUCKET REGION e.g. us-central1

# do not change these
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '2.1'

In [3]:
%%bash
gcloud  config  set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


We will use this helper function to write lists containing article ids, categories, and authors for each article in our database to local file. 

In [4]:
def write_list_to_disk(my_list, filename):
  with open(filename, 'w') as f:
    for item in my_list:
        line = "%s\n" % item
        f.write(line)

### Pull data from BigQuery

The cell below creates a local text file containing all the article ids (i.e. 'content ids') in the dataset. 

Have a look at the original dataset in [BigQuery](https://console.cloud.google.com/bigquery?p=cloud-training-demos&d=GA360_test&t=ga_sessions_sample). Then read through the query below and make sure you understand what it is doing. 

In [5]:
sql="""
#standardSQL

SELECT  
  (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) AS content_id 
FROM `cloud-training-demos.GA360_test.ga_sessions_sample`,   
  UNNEST(hits) AS hits
WHERE 
  # only include hits on pages
  hits.type = "PAGE"
  AND (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
GROUP BY
  content_id
  
"""

content_ids_list = bigquery.Client().query(sql).to_dataframe()['content_id'].tolist()
write_list_to_disk(content_ids_list, "content_ids.txt")
print("Some sample content IDs {}".format(content_ids_list[:3]))
print("The total number of articles is {}".format(len(content_ids_list)))

Some sample content IDs ['299965853', '299972248', '299410466']
The total number of articles is 15634


There should be 15,634 articles in the database.  
Next, we'll create a local file which contains a list of article categories and a list of article authors.

Note the change in the index when pulling the article category or author information. Also, we are using the first author of the article to create our author list.  
Refer back to the original dataset, use the `hits.customDimensions.index` field to verify the correct index.	 

In [6]:
sql="""
#standardSQL
SELECT  
  (SELECT MAX(IF(index=7, value, NULL)) FROM UNNEST(hits.customDimensions)) AS category  
FROM `cloud-training-demos.GA360_test.ga_sessions_sample`,   
  UNNEST(hits) AS hits
WHERE 
  # only include hits on pages
  hits.type = "PAGE"
  AND (SELECT MAX(IF(index=7, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
GROUP BY   
  category
"""
categories_list = bigquery.Client().query(sql).to_dataframe()['category'].tolist()
write_list_to_disk(categories_list, "categories.txt")
print(categories_list)

['News', 'Stars & Kultur', 'Lifestyle']


The categories are 'News', 'Stars & Kultur', and 'Lifestyle'.  
When creating the author list, we'll only use the first author information for each article.  

In [7]:
sql="""
#standardSQL
SELECT
  REGEXP_EXTRACT((SELECT MAX(IF(index=2, value, NULL)) FROM UNNEST(hits.customDimensions)), r"^[^,]+")  AS first_author  
FROM `cloud-training-demos.GA360_test.ga_sessions_sample`,   
  UNNEST(hits) AS hits
WHERE 
  # only include hits on pages
  hits.type = "PAGE"
  AND (SELECT MAX(IF(index=2, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
GROUP BY   
  first_author
"""
authors_list = bigquery.Client().query(sql).to_dataframe()['first_author'].tolist()
write_list_to_disk(authors_list, "authors.txt")
print("Some sample authors {}".format(authors_list[:10]))
print("The total number of authors is {}".format(len(authors_list)))

Some sample authors ['Alexander Huber', 'Christine Klafl', 'Philipp Albrechtsberger', 'Wolfgang Atzenhofer', 'Marlene Patsalidis', 'Peter Temel', 'Stefan Hofer', 'Michael Pammesberger', 'Helmut Brandstätter', 'Martina Salomon']
The total number of authors is 385


There should be 385 authors in the database. 

### Create train and test sets.

In this section, we will create the train/test split of our data for training our model. We use the concatenated values for visitor id and content id to create a farm fingerprint, taking approximately 90% of the data for the training set and 10% for the test set.

In [8]:
sql="""
WITH site_history as (
  SELECT
      fullVisitorId as visitor_id,
      (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) AS content_id,
      (SELECT MAX(IF(index=7, value, NULL)) FROM UNNEST(hits.customDimensions)) AS category, 
      (SELECT MAX(IF(index=6, value, NULL)) FROM UNNEST(hits.customDimensions)) AS title,
      (SELECT MAX(IF(index=2, value, NULL)) FROM UNNEST(hits.customDimensions)) AS author_list,
      SPLIT(RPAD((SELECT MAX(IF(index=4, value, NULL)) FROM UNNEST(hits.customDimensions)), 7), '.') as year_month_array,
      LEAD(hits.customDimensions, 1) OVER (PARTITION BY fullVisitorId ORDER BY hits.time ASC) as nextCustomDimensions
  FROM 
    `cloud-training-demos.GA360_test.ga_sessions_sample`,   
     UNNEST(hits) AS hits
   WHERE 
     # only include hits on pages
      hits.type = "PAGE"
      AND
      fullVisitorId IS NOT NULL
      AND
      hits.time != 0
      AND
      hits.time IS NOT NULL
      AND
      (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
)
SELECT
  visitor_id,
  content_id,
  category,
  REGEXP_REPLACE(title, r",", "") as title,
  REGEXP_EXTRACT(author_list, r"^[^,]+") as author,
  DATE_DIFF(DATE(CAST(year_month_array[OFFSET(0)] AS INT64), CAST(year_month_array[OFFSET(1)] AS INT64), 1), DATE(1970,1,1), MONTH) as months_since_epoch,
  (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(nextCustomDimensions)) as next_content_id
FROM
  site_history
WHERE (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(nextCustomDimensions)) IS NOT NULL
      AND ABS(MOD(FARM_FINGERPRINT(CONCAT(visitor_id, content_id)), 10)) < 9
"""
training_set_df = bigquery.Client().query(sql).to_dataframe()
training_set_df.to_csv('training_set.csv', header=False, index=False, encoding='utf-8')
training_set_df.head()

,visitor_id,content_id,category,title,author,months_since_epoch,next_content_id
0,1054303894414213464,299804373,Lifestyle,Bloggerin wegen Anstiftung zur Magersucht ange...,Elisabeth Mittendorfer,574,299937546
1,1054303894414213464,299437612,Lifestyle,Wenn Omi wieder nach der Liebe sucht,Yvonne Widler,574,299437612
2,1087837873805535641,299982579,News,VIDEO: Basejumper springen von Berg in Flugzeug,Mathias Kainz,574,187077794
3,1089650513224730843,299781837,News,Mordalarm in Wien: 31 Jahre alte Frau erstochen,None,574,299825001
4,1089650513224730843,299825001,News,Auslieferung: Mutmaßlicher Sechsfach-Mörder ha...,Michaela Reibenwein,574,299836800


In [9]:
sql="""
WITH site_history as (
  SELECT
      fullVisitorId as visitor_id,
      (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) AS content_id,
      (SELECT MAX(IF(index=7, value, NULL)) FROM UNNEST(hits.customDimensions)) AS category, 
      (SELECT MAX(IF(index=6, value, NULL)) FROM UNNEST(hits.customDimensions)) AS title,
      (SELECT MAX(IF(index=2, value, NULL)) FROM UNNEST(hits.customDimensions)) AS author_list,
      SPLIT(RPAD((SELECT MAX(IF(index=4, value, NULL)) FROM UNNEST(hits.customDimensions)), 7), '.') as year_month_array,
      LEAD(hits.customDimensions, 1) OVER (PARTITION BY fullVisitorId ORDER BY hits.time ASC) as nextCustomDimensions
  FROM 
    `cloud-training-demos.GA360_test.ga_sessions_sample`,   
     UNNEST(hits) AS hits
   WHERE 
     # only include hits on pages
      hits.type = "PAGE"
      AND
      fullVisitorId IS NOT NULL
      AND
      hits.time != 0
      AND
      hits.time IS NOT NULL
      AND
      (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
)
SELECT
  visitor_id,
  content_id,
  category,
  REGEXP_REPLACE(title, r",", "") as title,
  REGEXP_EXTRACT(author_list, r"^[^,]+") as author,
  DATE_DIFF(DATE(CAST(year_month_array[OFFSET(0)] AS INT64), CAST(year_month_array[OFFSET(1)] AS INT64), 1), DATE(1970,1,1), MONTH) as months_since_epoch,
  (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(nextCustomDimensions)) as next_content_id
FROM
  site_history
WHERE (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(nextCustomDimensions)) IS NOT NULL
      AND ABS(MOD(FARM_FINGERPRINT(CONCAT(visitor_id, content_id)), 10)) >= 9
"""
test_set_df = bigquery.Client().query(sql).to_dataframe()
test_set_df.to_csv('test_set.csv', header=False, index=False, encoding='utf-8')
test_set_df.head()

,visitor_id,content_id,category,title,author,months_since_epoch,next_content_id
0,1112146297759656551,299915364,News,Glyphosat-Verlängerung: SPD wirft CDU Vertraue...,None,574,299836255
1,1134243174384296784,299964154,News,Neue Seidenstraße: Ein chinesischer Keil in Eu...,Hermann Sileitsch-Parzer,574,299933565
2,1153984497631281150,299800661,Stars & Kultur,Meghan Markle im Porträt - Vom TV-Sternchen zu...,Christina Michlits,574,299906166
3,13826028694672839081,299836800,News,Protest: Gesichtsscanner in Apotheke abgedreht,Wolfgang Atzenhofer,574,299866366
4,1444851807585783223,299934703,News,Verkehrsunfall im Mühlviertel ließ Autoschiebe...,None,574,299930679


Let's have a look at the two csv files we just created containing the training and test set. We'll also do a line count of both files to confirm that we have achieved an approximate 90/10 train/test split.  
In the next notebook, **Content Based Filtering** we will build a model to recommend an article given information about the current article being read, such as the category, title, author, and publish date. 

In [10]:
%%bash
wc -l *_set.csv

   25599 test_set.csv
  232308 training_set.csv
  257907 total


In [11]:
!head *_set.csv

==> test_set.csv <==
1112146297759656551,299915364,News,Glyphosat-Verlängerung: SPD wirft CDU Vertrauensbruch vor,,574,299836255
1134243174384296784,299964154,News,Neue Seidenstraße: Ein chinesischer Keil in Europa,Hermann Sileitsch-Parzer,574,299933565
1153984497631281150,299800661,Stars & Kultur,Meghan Markle im Porträt - Vom TV-Sternchen zur Prinzessin,Christina Michlits,574,299906166
13826028694672839081,299836800,News,Protest: Gesichtsscanner in Apotheke  abgedreht,Wolfgang Atzenhofer,574,299866366
1444851807585783223,299934703,News,Verkehrsunfall im Mühlviertel ließ Autoschieber-Bande auffliegen,,574,299930679
1444851807585783223,299934703,News,Verkehrsunfall im Mühlviertel ließ Autoschieber-Bande auffliegen,,574,299181999
1563082078176046230,299772450,Stars & Kultur,"Kritik an Prinz Harry: ""Ein verzogener Rotzlöffel""",Elisabeth Spitzer,574,299800661
1729486252465286166,175262203,Lifestyle,Tansania: Die Hilfsprojekte von Christine Wallner,,552,175262218
1859931483041799173,2999